In [13]:
import boto3
import dask.dataframe as dd
from sagemaker import get_execution_role
import pandas as pd

from fastparquet import ParquetFile


role = get_execution_role()
bucket='tally-ai-dspt3'
folder = 'yelp-kaggle-raw-data'

pd.set_option('display.max_columns', None)

print(f"S3 Bucket is {bucket}, and Folder is {folder}")

S3 Bucket is tally-ai-dspt3, and Folder is yelp-kaggle-raw-data


In [2]:
# follow the following steps below verbatim and open in terminal to run them except for kernel restart which is at menu
# source activate python3
# conda install dask -y
# conda install s3fs -c conda-forge -y
# restart kernel
#Note, to run parquet which is a way to export to S3 with highly reduces dask files, you need to add the following command:
# conda install -c conda-forge fastparquet
#or togeher: conda install s3fs fastparquet -c conda-forge -y

# Loading Yelp Businees `attributes`

In [14]:
data_key = 'yelp_academic_dataset_business.json'
data_location = 's3://{}/{}/{}'.format(bucket, folder, data_key)
business = dd.read_json(data_location, lines=True)
business.head(5)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,10913 Bailey Rd,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...",f9NumwFMBDn751xgFiRbNA,"Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...",Cornelius,"{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'...",1,35.462724,-80.852612,The Range At Lake Norman,28031,36,3.5,NC
1,"8880 E Via Linda, Ste 107","{'GoodForKids': 'True', 'ByAppointmentOnly': '...",Yzvjg0SayhoZgCljUJRF9Q,"Health & Medical, Fitness & Instruction, Yoga,...",Scottsdale,None,1,33.569404,-111.890264,"Carlos Santo, NMD",85258,4,5.0,AZ
2,3554 Rue Notre-Dame O,None,XNoUzKckATkOD1hP6vghZg,"Pets, Pet Services, Pet Groomers",Montreal,None,1,45.479984,-73.580070,Felinus,H4C 1P4,5,5.0,QC
3,1015 Sharp Cir,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",6OAZjbxqM5ol29BuHsil3w,"Hardware Stores, Home Services, Building Suppl...",North Las Vegas,"{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ...",0,36.219728,-115.127725,Nevada House of Hose,89030,3,2.5,NV
4,4827 E Downing Cir,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",51M2Kk903DFYI6gnB5I6SQ,"Home Services, Plumbing, Electricians, Handyma...",Mesa,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '...",1,33.428065,-111.726648,USE MY GUY SERVICES LLC,85205,26,4.5,AZ


In order to use dask

1. Activate Conda Python 3 `source activate python3`
2. Install dask and s3fs `conda install dask s3fs -c conda-forge -y`

# Loading Yelp Businees `reviews` ~ 6GB data

In [5]:
#uncomment below to spin reviews which will be too large. Suggested we create a subset on RDS/postgres and then spin only what's necessary
data_key = 'yelp_academic_dataset_review.json'
data_location = 's3://{}/{}/{}'.format(bucket, folder, data_key)
reviews = dd.read_json(data_location, blocksize=32e6)

In [15]:
reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,-MhfebM0QIsKt87iDN-FNw,0,2015-04-15 05:21:16,0,xQY8N_XvtGbearJ5X4QryQ,2,"As someone who has worked with many museums, I...",5,OwjRMXRC0KyPrIlcjaXeFQ
1,lbrU8StCq3yDfr-QMnGrmQ,0,2013-12-07 03:16:52,1,UmFMZ8PyXZTY2QcwzsfQYA,1,I am actually horrified this place is still in...,1,nIJD_7ZXHq-FX8byPMOkMQ
2,HQl28KMwrEKHqhFrrDqVNQ,0,2015-12-05 03:18:11,0,LG2ZaYiOgpr2DK_90pYjNw,5,I love Deagan's. I do. I really do. The atmosp...,1,V34qejxNsCbcgD8C0HVk-Q
3,5JxlZaqCnk1MnbgRirs40Q,0,2011-05-27 05:30:52,0,i6g_oA9Yf9Y31qt0wibXpw,1,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",0,ofKDkJKXSKZXu5xJNGiiBQ
4,IS4cv902ykd8wj1TR0N3-A,0,2017-01-14 21:56:57,0,6TdNDKywdbjoTkizeMce8A,4,"Oh happy day, finally have a Canes near my cas...",0,UgMW8bLE0QMJDCkQ1Ax5Mg


In [7]:
#need to join the three datasets together. Dask join is just like dask
#once joined, filter with restaurants
#once filtered, only have restaurants in database, export as .csv
#once have .csv file load to S3 bucket

In [7]:
#uncomment below to spin reviews which will be too large. Suggested we create a subset on RDS/postgres and then spin only what's necessary
data_key = 'yelp_academic_dataset_user.json'
data_location = 's3://{}/{}/{}'.format(bucket, folder, data_key)
users = dd.read_json(data_location, blocksize=32e6)

In [16]:
users.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,3.57,22,0,22,3,1,2,11,0,15,1,10,227,,14,"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg...",225,Rafael,553,628,ntlvfPzc8eglqvk92iDIAw,2007-07-06 03:27:11
1,3.84,63,2,63,36,1,4,33,5,37,5,21,400,"2008,2009,2010,2011,2012,2013",27,"ly7EnE8leJmyqyePVYFlug, pRlR63iDytsnnniPb3AOug...",316,Michelle,564,790,FOBRPlBHa3WPHFB5qYDlVg,2008-04-28 01:29:25
2,3.44,17,1,17,9,0,6,3,1,7,0,4,103,2010,5,"Uwlk0txjQBPw_JhHsQnyeg, Ybxr1tSCkv3lYA0I1qmnPQ...",125,Martin,60,151,zZUnPeh2hEp0WydbAZEOOg,2008-08-28 23:40:05
3,3.08,7,0,7,2,0,1,7,0,14,0,2,84,2009,6,"iog3Nyg1i4jeumiTVG_BSA, M92xWY2Vr9w0xoH8bPplfQ...",160,John,206,233,QaELAmRcDc5TfJEylaaP8g,2008-09-20 00:08:14
4,4.37,31,1,31,8,1,9,22,31,28,2,19,512,"2009,2010,2011,2012,2014,2015,2016,2017,2018",78,"3W3ZMSthojCUirKEqAwGNw, eTIbuu23j9tOgmIa9POyLQ...",400,Anne,485,1265,xvu8G900tezTzbbfqmTKvA,2008-08-09 00:30:27


In [10]:
#business_reviews = business.merge(reviews, on = 'business_id', how = 'inner')

In [11]:
#business_reviews.head()

In [12]:
#business_reviews_users = business_reviews.merge(users, on = 'user_id', how = 'inner')

In [13]:
#business_reviews_users.head()

In [14]:
#method to pull unique values out of column 'category' in business
# def unique_col(col):
#     return ','.join(set(col.split(',')))

# x = business.categories.apply(unique_col)

In [9]:
#drop na values
business2 = business.dropna(subset=['categories'])

In [10]:
restaurants = business2[business2['categories'].str.contains('Restaurants')]

In [12]:
restaurants.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
8,404 E Green St,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...",pQeaRpvuhoEqudo3uymHIQ,"Ethnic Food, Food Trucks, Specialty Food, Impo...",Champaign,"{'Monday': '11:30-14:30', 'Tuesday': '11:30-14...",1,40.110446,-88.233073,The Empanadas House,61820,5,4.5,IL
20,4508 E Independence Blvd,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...",CsLQLiRoafpJPJSkNX2h5Q,"Food, Restaurants, Grocery, Middle Eastern",Charlotte,None,0,35.194894,-80.767442,Middle East Deli,28205,5,3.0,NC
24,"15480 Bayview Avenue, unit D0110","{'RestaurantsTableService': 'False', 'Restaura...",eBEfgOPG7pvFhb2wcG9I7w,"Restaurants, Cheesesteaks, Poutineries",Aurora,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",1,44.010962,-79.448677,Philthy Phillys,L4G 7J1,4,4.5,ON
25,300 John Street,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",lu7vtrp_bE9PnxWfA8g4Pg,"Japanese, Fast Food, Food Court, Restaurants",Thornhill,None,1,43.820492,-79.398466,Banzai Sushi,L3T 5W4,7,4.5,ON
30,13071 Yonge Street,"{'Ambience': '{'touristy': False, 'hipster': F...",9sRGfSVEfLhN_km60YruTA,"Persian/Iranian, Turkish, Middle Eastern, Rest...",Richmond Hill,"{'Tuesday': '12:0-21:0', 'Wednesday': '12:0-21...",1,43.947011,-79.454862,Apadana Restaurant,L4E 1A5,3,3.0,ON


In [25]:
#stars appear as a column in the first two datasets, to idenity, renaming
restaurants['stars_business'] = restaurants['stars']
restaurants2 = restaurants.drop(['stars'], axis=1)

In [36]:
#doing same for reviews:
reviews['stars_reviews'] = reviews['stars']
reviews2 = reviews.drop(['stars'], axis=1)

In [37]:
restaurants2_reviews2 = restaurants2.merge(reviews2, on = 'business_id', how = 'inner')

In [38]:
restaurants2.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'state', 'stars_business'],
      dtype='object')

In [39]:
reviews.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id', 'stars_review', 'stars_reviews'],
      dtype='object')

In [40]:
restaurants2_reviews2.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'state', 'stars_business', 'cool', 'date', 'funny',
       'review_id', 'text', 'useful', 'user_id', 'stars_review',
       'stars_reviews'],
      dtype='object')

In [41]:
restaurants2_reviews2.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,state,stars_business,cool,date,funny,review_id,text,useful,user_id,stars_review,stars_reviews
0,404 E Green St,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...",pQeaRpvuhoEqudo3uymHIQ,"Ethnic Food, Food Trucks, Specialty Food, Impo...",Champaign,"{'Monday': '11:30-14:30', 'Tuesday': '11:30-14...",1,40.110446,-88.233073,The Empanadas House,61820,5,IL,4.5,0,2013-04-11 18:36:15,0,2TtLt9skrBwBM6QGEhFYCg,I love the empanadas from the Empanadas House!...,0,eSQ3z93DlzkpXK_H6MFEMw,5,5
1,4508 E Independence Blvd,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...",CsLQLiRoafpJPJSkNX2h5Q,"Food, Restaurants, Grocery, Middle Eastern",Charlotte,None,0,35.194894,-80.767442,Middle East Deli,28205,5,NC,3.0,0,2010-03-13 13:54:30,0,XyHl5E80hAAlkhOt8m4Aww,"Definitely under new management, and the dinin...",5,whqfWVBj4x5EDboIVDOymQ,3,3
2,9595 W Tropicana Ave,"{'Alcohol': 'u'none'', 'WiFi': 'u'no'', 'GoodF...",fnZrZlqW1Z8iWgTVDfv_MA,"Mexican, Restaurants, Fast Food",Las Vegas,None,0,36.099738,-115.301568,Carl's Jr,89147,15,NV,2.5,0,2010-11-27 09:33:34,0,WUAt5-krh075Ie44np8Hew,So why would I be giving a Fast Food chain loc...,0,YDkC5VVT8s9NlIZtAr8NUA,5,5
3,843 Kipling Avenue,"{'RestaurantsPriceRange2': '2', 'RestaurantsAt...",rVBPQdeayMYht4Uv_FOLHg,"Restaurants, Burgers, Food",Toronto,None,0,43.633291,-79.531768,Gourmet Burger Company,M8Z 5G9,13,ON,3.0,0,2011-07-29 16:00:02,1,OobdounFYjkAbIDsVUNwRw,"We've eaten here a few times now, and have bee...",0,zsZVg16yjZu5NIiS0ayjrQ,5,5
4,5252 Rue Jean Talon O,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...",LoRef3ChgZKbxUio-sHgQg,"Restaurants, Lebanese, Middle Eastern",Montréal,None,1,45.494870,-73.651904,Amir,H4P 2A7,18,QC,3.0,0,2013-05-29 16:24:17,0,2L1UbgcrQSTNObqZEufKKw,"Good food, terrible customer service. For me ...",1,schfOYW71VamTTdfzqDlXQ,2,2


In [31]:
users.columns

Index(['average_stars', 'compliment_cool', 'compliment_cute',
       'compliment_funny', 'compliment_hot', 'compliment_list',
       'compliment_more', 'compliment_note', 'compliment_photos',
       'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool',
       'elite', 'fans', 'friends', 'funny', 'name', 'review_count', 'useful',
       'user_id', 'yelping_since'],
      dtype='object')

In [32]:
restaurants2_reviews2.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'state', 'stars_business', 'cool', 'date', 'funny',
       'review_id', 'text', 'useful', 'user_id', 'stars_review'],
      dtype='object')

In [33]:
restaurants2_reviews2_users = restaurants2_reviews2.merge(users, on = 'user_id', how = 'inner')

In [35]:
restaurants2_reviews2_users.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name_x', 'postal_code',
       'review_count_x', 'state', 'stars_business', 'cool_x', 'date',
       'funny_x', 'review_id', 'text', 'useful_x', 'user_id', 'stars_review',
       'average_stars', 'compliment_cool', 'compliment_cute',
       'compliment_funny', 'compliment_hot', 'compliment_list',
       'compliment_more', 'compliment_note', 'compliment_photos',
       'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool_y',
       'elite', 'fans', 'friends', 'funny_y', 'name_y', 'review_count_y',
       'useful_y', 'yelping_since'],
      dtype='object')

In [43]:
#doing same for reviews:
restaurants2_reviews2['name_reviews'] = restaurants2_reviews2['name']
restaurants2_reviews2['review_count_reviews'] = restaurants2_reviews2['review_count']
restaurants2_reviews2['cool_reviews'] = restaurants2_reviews2['cool']
restaurants2_reviews2['funny_reviews'] = restaurants2_reviews2['funny']
restaurants2_reviews2['useful_reviews'] = restaurants2_reviews2['useful']
restaurants3_reviews3 = restaurants2_reviews2.drop(['name', 'review_count', 'cool', 'funny', 'useful'], axis=1)

In [44]:
#now do this again to users:
#doing same for reviews:
users['name_users'] = users['name']
users['review_count_users'] = users['review_count']
users['cool_users'] = users['cool']
users['funny_users'] = users['funny']
users['useful_users'] = users['useful']
users2 = users.drop(['name', 'review_count', 'cool', 'funny', 'useful'], axis=1)

In [ ]:
#make sure to include: address, hours, postal code, and date

In [22]:
#restaurants2_reviews_users.head()

In [45]:
restaurants3_reviews3_users2 = restaurants3_reviews3.merge(users2, on = 'user_id', how = 'inner')

In [46]:
#check that everything worked:
restaurants3_reviews3_users2.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'postal_code', 'state',
       'stars_business', 'date', 'review_id', 'text', 'user_id',
       'stars_review', 'stars_reviews', 'name_reviews', 'review_count_reviews',
       'cool_reviews', 'funny_reviews', 'useful_reviews', 'average_stars',
       'compliment_cool', 'compliment_cute', 'compliment_funny',
       'compliment_hot', 'compliment_list', 'compliment_more',
       'compliment_note', 'compliment_photos', 'compliment_plain',
       'compliment_profile', 'compliment_writer', 'elite', 'fans', 'friends',
       'yelping_since', 'name_users', 'review_count_users', 'cool_users',
       'funny_users', 'useful_users'],
      dtype='object')

In [51]:
final_combined = restaurants3_reviews3_users2.drop(['attributes', 'hours'], axis=1)

In [31]:
#reducing file size further by cropping out additional columns for parquet to pass, as issues with RAM cause kernel to restart
#restaurants_reviews_users2 = restaurants_reviews_users.drop(['address', 'attributes', 'hours', 'is_open', 'postal_code', 'date', 
#                                                              'friends', 'yelping_since','compliment_cool', 'compliment_cute', 'compliment_hot', 
#                                                              'compliment_list', 'compliment_more', 'compliment_note', 'compliment_photos', 
#                                                              'compliment_plain', 'compliment_profile'], axis=1)

In [48]:
final_combined.head()

,address,business_id,categories,city,hours,is_open,latitude,longitude,postal_code,state,stars_business,date,review_id,text,user_id,stars_review,stars_reviews,name_reviews,review_count_reviews,cool_reviews,funny_reviews,useful_reviews,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,elite,fans,friends,yelping_since,name_users,review_count_users,cool_users,funny_users,useful_users
0,"3101 Central Ave, Ste 1",zRr45TrTgq8XAKwvjM0W_Q,"Restaurants, Chinese",Charlotte,None,1,35.217665,-80.792571,28205,NC,4.5,2013-05-27 16:04:09,eAeFc3zZE30oJU7_1DmpPw,Four Stars for the take out duck! Thanks Bruc...,ZvmC522FAwxt8qT1jxqliw,4,4,Vanloi Chinese Barbecue Restaurant,7,2,1,4,2.68,16,0,16,4,0,5,48,11,29,0,36,,11,EW4r-ipoq_nm2JLSF7nILA,2010-10-10 16:05:18,George,267,196,254,731
1,"2233 Matthews Township Pkwy, Ste E",TTFbaLpIwrjFx7YIL2pbJA,"Middle Eastern, Restaurants, Mediterranean, Ve...",Matthews,"{'Tuesday': '11:0-21:30', 'Wednesday': '11:0-2...",1,35.128190,-80.702241,28105,NC,4.5,2013-07-28 22:26:47,cYpx46m00J0RYteQsxE4Gg,Nearly ready for prime time. \n\nThis is no fa...,ZvmC522FAwxt8qT1jxqliw,3,3,Kabab-Je Rotisserie & Grille,557,4,0,7,2.68,16,0,16,4,0,5,48,11,29,0,36,,11,EW4r-ipoq_nm2JLSF7nILA,2010-10-10 16:05:18,George,267,196,254,731
2,"7741 Colony Rd, Ste A4",vSB8Yrz0KslzqnlPEBfqNA,"Taiwanese, Chinese, Restaurants",Charlotte,None,0,35.106970,-80.806871,28226,NC,4.0,2013-07-29 20:04:11,s4dUwSJsZyLoo4xDz_8r0w,CLOSED DOWN\n-------------------------\nOnce a...,ZvmC522FAwxt8qT1jxqliw,4,4,Tomi Chinese Restaurant,9,0,2,4,2.68,16,0,16,4,0,5,48,11,29,0,36,,11,EW4r-ipoq_nm2JLSF7nILA,2010-10-10 16:05:18,George,267,196,254,731
3,10400 E Independence Blvd.,SQEA7XcYZ6ygpuSnCSy1IQ,"Restaurants, American (Traditional), Barbeque,...",Matthews,"{'Monday': '16:0-22:0', 'Tuesday': '16:0-22:0'...",1,35.128536,-80.708933,28105,NC,3.5,2013-01-25 02:07:10,-3TvAkdBPRaQKAxn-myiog,I reached corporate back then and they were re...,ZvmC522FAwxt8qT1jxqliw,3,3,Texas Roadhouse,172,1,1,1,2.68,16,0,16,4,0,5,48,11,29,0,36,,11,EW4r-ipoq_nm2JLSF7nILA,2010-10-10 16:05:18,George,267,196,254,731
4,7828 Rea Rd,3OKwP3ylofhDJDtWJQu5GQ,"Mediterranean, Restaurants",Charlotte,"{'Tuesday': '11:30-22:0', 'Wednesday': '11:30-...",0,35.061110,-80.817254,28277,NC,3.5,2012-12-01 17:42:38,pT2FEqeNOff-HMZwsSlI4g,This was a weeks ago but I realize I won't be ...,ZvmC522FAwxt8qT1jxqliw,2,2,Kabob Grill,86,0,1,1,2.68,16,0,16,4,0,5,48,11,29,0,36,,11,EW4r-ipoq_nm2JLSF7nILA,2010-10-10 16:05:18,George,267,196,254,731


In [26]:
# 'compliment_cool', 'compliment_cute', 'compliment_hot', 'compliment_list', 'compliment_more', 'compliment_note', 'compliment_photos', 'compliment_plain', 'compliment_profile', 'compliment_write',

In [27]:
#add the following code to run parquet, in terminal:
#conda install -c conda-forge fastparquet

In [52]:
final_combined.to_parquet('s3://tally-ai-dspt3/yelp-kaggle-raw-data/final_combined.parquet.gzip', compression='gzip') 
             

In [30]:
#dd.to_csv(restaurants_reviews_users,'s3://tally-ai-dspt3/yelp-kaggle-raw-data/restaurants_reviews_users.csv')